In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load dataset
dataset, info = tfds.load('malaria', as_supervised=True, with_info=True)
data = dataset['train']  # There is only 'train' split in Malaria dataset

# Split manually: 80% for training, 20% for testing
train_size = int(0.8 * info.splits['train'].num_examples)
test_size = int(0.2 * info.splits['train'].num_examples)

train_data = data.take(train_size)
test_data = data.skip(train_size)

print("✅ Dataset Loaded and Split Successfully!")
print("Training samples:", train_size)
print("Testing samples:", test_size)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/malaria/incomplete.51ARSS_1.0.0/malaria-train.tfrecord*...:   0%|         …

Dataset malaria downloaded and prepared to /root/tensorflow_datasets/malaria/1.0.0. Subsequent calls will reuse this data.
✅ Dataset Loaded and Split Successfully!
Training samples: 22046
Testing samples: 5511


In [ ]:
IMG_SIZE = 128
BATCH_SIZE = 32

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train = train_data.map(preprocess).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test = test_data.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train, epochs=10, validation_data=test)

Epoch 1/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 41s 50ms/step - accuracy: 0.6834 - loss: 0.5579 - val_accuracy: 0.9438 - val_loss: 0.1725
Epoch 2/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 29s 42ms/step - accuracy: 0.9485 - loss: 0.1656 - val_accuracy: 0.9494 - val_loss: 0.1542
Epoch 3/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 42s 42ms/step - accuracy: 0.9566 - loss: 0.1335 - val_accuracy: 0.9514 - val_loss: 0.1542
Epoch 4/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 48s 53ms/step - accuracy: 0.9593 - loss: 0.1226 - val_accuracy: 0.9585 - val_loss: 0.1403
Epoch 5/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 29s 42ms/step - accuracy: 0.9651 - loss: 0.1073 - val_accuracy: 0.9572 - val_loss: 0.1418
Epoch 6/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 30s 43ms/step - accuracy: 0.9651 - loss: 0.0951 - val_accuracy: 0.9554 - val_loss: 0.1606
Epoch 7/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 41s 43ms/step - accuracy: 0.9706 - loss: 0.0838 - val_accuracy: 0.9556 - val_loss: 0.1472
Epoch 8/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 38s 54ms/step - accuracy: 0.9747 - loss: 0.0742 - 

In [ ]:
test_loss, test_acc = model.evaluate(test)
print(f"\n✅ Test Accuracy: {test_acc*100:.2f}%")

173/173 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.9601 - loss: 0.1709

✅ Test Accuracy: 95.48%


In [ ]:
model.save('malaria_cnn_model.h5')
print("✅ Model saved successfully as malaria_cnn_model.h5")

✅ Model saved successfully as malaria_cnn_model.h5
